In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import random 
import scipy.stats as stats
import matplotlib.pyplot as plt
import sys, os
import warnings
import processing_module as pm
import scipy.stats as stats
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [ ]:
def load_graph_pr(blocks: int, folder_name: str, gen_name: str) -> list:
    """
    Загрузка цепочек данных по пейджрангам по определённым блокам.
    """
    for t in range(0, blocks):
        my_path = os.path.abspath("") + "/.."
        CA_graph_df = pd.read_csv(my_path + "/Data_postproc/" + folder_name + "/Graph" + gen_name \
                                                   + str(t) + "_part.csv")
        G_init = nx.from_pandas_edgelist(CA_graph_df)
        pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]
    return pr


# def load_graph(blocks: int, folder_name: str, gen_name: str) -> nx.Graph:
#     """
#     Загрузка графа.
#     """
#     for t in range(0, blocks):
#         my_path = os.path.abspath("") + "/.."
#         CA_graph_df = pd.read_csv(my_path + "/Data_postproc/" + folder_name + "/Graph" + gen_name \
#                                                    + str(t) + "_part.csv")
#         G_init = nx.from_pandas_edgelist(CA_graph_df)
#         pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]
#     return pr


def load_graph_di(t: int, folder_name: str, gen_name: str) -> nx.DiGraph:
    """
    Загрузка направленного графа.
    """
    my_path = os.path.abspath("") + "/.."
    CA_graph_df = pd.read_csv(my_path + "/Data_postproc/" + folder_name + "/Graph" + gen_name \
                                                   + str(t) + "_part.csv")
    G_init = nx.from_pandas_edgelist(CA_graph_df, source='source', target='target', \
                                   create_using=nx.DiGraph())
    return G_init

<span style="color:orange">
    
## Graph analysis CA + PA ##

In [11]:
print(len(pr[19]))

NameError: name 'pr' is not defined

In [ ]:
x_gr = list()
y_gr = list()
for i in range(blocks):
    x = pm.hill(pr[i])
    eye_val = (pm.eye_ball(x))[0]
    x_gr.append(i)
    y_gr.append(eye_val)
    print(i, eye_val)

In [ ]:
plt.plot(x_gr, y_gr, marker='o', markersize=7)
plt.xlabel('Временной шаг') #Хвостовые ингде
plt.ylabel('EVI') #Подпись для оси y
plt.title('Зависимость EVI от временного шага') #Название
plt.show()

In [ ]:
med = []
for i in range(30):
    med.append(np.median(pr[i]))

plt.plot(x_gr, med, marker='o', markersize=7)
plt.xlabel('Временной блок') #Хвостовые ингде
plt.ylabel('Медиана данных в блоке') #Подпись для оси y
plt.title('Медиана данных в блоке от времени') #Название
plt.show()

In [ ]:
def phillips_loretan(x1, x2):
    """
    Функция теста phillips_loretan для двух последовательностей
    """
    tail_ind_1, tail_num_1 = pm.eye_ball(pm.hill(x1))
    tail_ind_2, tail_num_2 = pm.eye_ball(pm.hill(x2))
    s = (tail_num_1*(tail_ind_2)**2 * (tail_ind_1/tail_ind_2 - 1)**2)/ \
    (tail_ind_1**2 + (tail_num_1/tail_num_2)*tail_ind_2**2)
    return s

In [12]:
"""
Тесты Филлипса Лоретана для разных блоков.
"""

group_1 = []
for i in range(0, 10):
    for j in range(0, 10):
        # print(i, j, phillips_loretan(pr[i], pr[j]))
        group_1.append(phillips_loretan(pr[i], pr[j]))

group_1_2 = []
# print('\n')
for i in range(0, 10):
    for j in range(10, 20):
        # print(i, j, phillips_loretan(pr[i], pr[j]))
        group_1_2.append(phillips_loretan(pr[i], pr[j]))

group_2 = []
# print('\n')
for i in range(10, 20):
    for j in range(10, 20):
        # print(i, j, phillips_loretan(pr[i], pr[j]))
        group_2.append(phillips_loretan(pr[i], pr[j]))

print('group_1 \n', min(group_1), max(group_1), '\n')
print('group_2 \n', min(group_2), max(group_2), '\n')
print('group_1_2 \n', min(group_1_2), max(group_1_2), '\n')

NameError: name 'phillips_loretan' is not defined

In [13]:
phillips_loretan(pr[14], pr[15])

NameError: name 'phillips_loretan' is not defined

<span style="color:orange">
    
## Graph analysis ABG ##

In [34]:
gen_name = '_ABG'
folder_name = "ABG_graph_3"
t = 3

G_ABG_1 = load_graph_di(t, folder_name, gen_name)

In [35]:
print(len(G_ABG_1.nodes()))

13622


In [36]:
print('graph analysis')
print(G_ABG_1)

# G_ABG_und = G_ABG_1.to_undirected()

comm = nx.community.louvain_communities(G_ABG_1, resolution = 1)
comm = sorted(comm, key=len, reverse=True)


print('\n \n \n degree analysis')
in_data = pd.DataFrame(data = G_ABG_1.in_degree(), columns=['node', 'node_degree'])
print(in_data.sort_values(by = 'node_degree', ascending = False)[0:20])


print('\n \n \n Comm len analysis')
print('all communities:', len(comm))
for i in range(10):
    # if (len(comm[i]) > 0.05 * len(G_ABG_1.nodes())):
    #     print(i, len(comm[i]))
    print(i, len(comm[i]))
# print(comm[1])

print(' \n \n \n Comm separeted analysis')
in_data = pd.DataFrame(data = G_ABG_1.in_degree(), columns=['node', 'node_degree'])
for i in range(5):
    ser_comm = pd.DataFrame(data=list(comm[i]), columns=['node'])
    print('\n', i)
    print(pd.merge(ser_comm, in_data).sort_values(by = 'node_degree', ascending = False)[0:5])

graph analysis
DiGraph with 13622 nodes and 23414 edges

 
 
 degree analysis
    node  node_degree
5      6         1318
1      2          751
3      3          542
0      1          495
12   166          216
20    74          213
4     25          166
6     42          163
19    50          140
14    88          106
2      0          100
13    47           98
22    64           95
11   164           91
40   150           86
32   294           69
64    96           57
30   514           52
33   417           49
27   408           49

 
 
 Comm len analysis
all communities: 4167
0 680
1 325
2 291
3 223
4 132
5 88
6 72
7 62
8 62
9 50
 
 
 
 Comm separeted analysis

 0
     node  node_degree
1       6         1318
178   543           11
519  9727           10
358  1025            9
318   922            8

 1
     node  node_degree
0       2          751
200   619           15
171  2599           12
207   651           12
142  1455           11

 2
     node  node_degree
0       3        

<span style="color:orange">
    
## Old code ##

In [ ]:
# for t in range(0, blocks//2):
#     G_init = gen_graph_CA(G_init, num_nodes, num_neigh)
#     pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]
#     (nx.to_pandas_edgelist(G_init)).to_csv(my_path + "/Data_postproc" + "/Big_graph" + "/CA_PA_" \
#                                                + str(t) + "_nodes" + ".csv", index=False)


# for t in range(0, blocks//2):
#     my_path = os.path.abspath("") + "/.."
#     CA_graph_df = pd.read_csv(my_path + "/Data_postproc" + "/Big_graph" + "/CA_PA_" \
#                                                + str(t) + "_nodes" + ".csv")
#     G_init = nx.from_pandas_edgelist(CA_graph_df)
#     pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]

In [6]:
# for i in range(len(edges_ca['source'])):
#     print(i, edges_ca[edges_ca['source'] == i].size)
# print(edges_ca[edges_ca['source'] == 1])

In [7]:
# data = {
#     'Source': [1, 2],
#     'Target': [2, 1],
#     'Node_degree': [1, 1]
# }

# row_labels = [1, 2]

# PA_graph = pd.DataFrame(data=data, index=row_labels)

# flg_PA = 0

# l = []
# if(flg_PA == 1):
#     for i in tqdm(range(3, 10000)):
#         k = (random.choices(PA_graph['Source'].values, weights=PA_graph['Node_degree'].values))[0]
#         if (i == 3):
#             l = [PA_graph['Source'].values, PA_graph['Node_degree'].values]
#         # print(k)
#         PA_graph.loc[i] = [i, k, 1]
#         PA_graph['Node_degree'][k] += 1

# print(l)

In [49]:
# генерация графа CA старая версия
# 
# if(flg_CA == 1):
#     for i in tqdm(range(4, num_nodes_ca)):
#         local_calst = list(nx.clustering(G_ca).values())
#         for j in range(len(local_calst)):
#             local_calst[j] += 0.001
#         k = []
#         for i1 in range(num_neigh):
#             k.append((random.choices(list(nx.clustering(G_ca).keys()), weights=local_calst))[0])
#         for i1 in range(num_neigh):
#             G_ca.add_edge(k[i1], i)


In [ ]:
# def gen_graph_ABG(G_input: nx.DiGraph, num_iterations: int, alpha: float, beta: float, \
#                   d_in: float, d_out: float) -> nx.DiGraph:
#     """
#     Функция генерации вершин графа методом альфа, бетта, гамма присоединения.
#     Используются библиотеки tqdm, networkx.
#     gamma = 1 - (alpha + beta).
    
#     Input:
#         G_input - изначальный ориентированный граф к которому присоединяются новые вершины.
#         num_iterations - количество итераций алгоритма.
#         alpha - float вероятность.
#         beta - float вероятность. 
#         d_in - float.
#         d_out - float.
        
#     Output:
#         Изначальный граф к которому присоеденены вершины.
#     """
#     assert ((alpha + beta) <= 1)

#     i_start = len(list(G_input.nodes))
#     gamma = 1 - alpha - beta
#     print(alpha, beta, gamma, G_input)

#     for i in tqdm(range(0, num_iterations + 1)):
#         iter_prob = random.choices([1, 2, 3], weights=[alpha, beta, gamma])[0]
        
#         I_n1 = list(dict(G_input.in_degree()).values())
#         O_n1 = list(dict(G_input.out_degree()).values())
        
#         N_I = list(dict(G_input.in_degree()).keys())
#         N_O = list(dict(G_input.out_degree()).keys())
        
#         N = len(I_n1)
#         n = sum(list(dict(G_input.in_degree()).values()))
#         u = list(G_input.nodes)[-1] + 1
#         num = len(I_n1) + 1

#         assert (N_I == N_O)
        
#         if (iter_prob == 1):
#             denominator = n - 1 + d_in * N
#             local_eq = []
#             for j in range(len(I_n1)):
#                 local_eq.append((I_n1[j] + d_in) / denominator)
#             w = (random.choices(N_I, weights=local_eq))[0]
#             G_input.add_edge(num, w)

#         if (iter_prob == 2):
#             denominator_1 = n - 1 + d_in * N
#             denominator_2 = n - 1 + d_out * N
#             local_eq = []
#             for j in range(len(I_n1)):
#                 local_eq.append(((I_n1[j] + d_in) / denominator_1) * ((O_n1[j] + d_out) / denominator_2))
#             w = (random.choices(N_I, weights=local_eq))[0]
#             u = (random.choices(N_I, weights=local_eq))[0]
#             G_input.add_edge(u, w)
        
#         if (iter_prob == 3):
#             denominator = n - 1 + d_out * N
#             local_eq = []
#             for j in range(len(O_n1)):
#                 local_eq.append((O_n1[j] + d_out) / denominator)
#             w = (random.choices(N_O, weights=local_eq))[0]
#             G_input.add_edge(w, num)

#     return G_input